# Betting Prediction Model - Scraping Premier League Table (Training Data)

## Overview

This notebook aims to scrape data from the Premier League table from sofascore to gather training data for our betting prediction model. The extracted data will be utilized to train our model for accurate predictions. The primary objectives of this notebook include:

1. **Data Extraction**: Retrieve relevant information from the Premier League table, including team standings, recent match results, and performance statistics.
2. **Data Processing**: Cleanse and format the scraped data to ensure consistency and accuracy for model training.
3. **Database Integration**: Store the processed data in the previous_league_standings table which we would later merge with completed_matches table and append to our `training_data` table within our database for future model training sessions.

## Steps Involved

1. **Environment Setup and Imports**: Import necessary libraries and configure settings.
2. **Web Scraping**: Utilize web scraping techniques to extract data from the Premier League table.
3. **Data Cleansing and Transformation**: Process the extracted data to ensure uniformity and reliability.
4. **Database Interaction**: Insert the processed data into the `training_data` table to build a comprehensive dataset.


## Requirements

- Python 3.x
- Libraries: `selenium web driver`, `pandas`, `sqlalchemy`
- Access to the database containing the `training_data` table

## Notes

- Prior to execution, ensure that database credentials are correctly configured.
- This notebook should be executed periodically to keep the completed_matches up-to-date with the previous Premier League data. to ensure this, run this notebook before you run `Betting Prediction Model - Scraping Forebet Prediction Table Completed Matches`

Let's commence by setting up our environment and importing essential libraries.


In [1]:
from selenium import webdriver
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd

In [10]:
PATH = 'C:/Users/kevin/Desktop/tools/chromedriver-win64/chromedriver'
driver = webdriver.Chrome(PATH) 
driver.get('https://www.sofascore.com/tournament/football/england/premier-league/17#id:52186');

C:\Users\kevin\AppData\Local\Temp\ipykernel_20600\4087939154.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [11]:
# Now find the fixture containers within the body element
league_table_results = driver.find_elements("class name", "eHXJll")
print(league_table_results)


[<selenium.webdriver.remote.webelement.WebElement (session="5d5883af6907c8a26a31ce5ba0f2d155", element="f.9109CED7DB784DA64C5D8EB5593345BA.d.8BB8B6CC1890623EF22C84038B94C402.e.86")>]


In [12]:
league_table_results_container = []

for results in league_table_results:
    # Extract the text from the fixture container
    results_text = results.text
    league_table_results_container.append(results_text)
    print(results_text)

ALLHOMEAWAY
#
Team
P
W
D
L
Goals
Last 5
PTS
1
Man City
38
28
7
3
96:34
W
W
W
W
W
91
2
Arsenal
38
28
5
5
91:29
W
W
W
W
W
89
3
Liverpool
38
24
10
4
86:41
L
D
W
D
W
82
4
Aston Villa
38
20
8
10
76:61
W
D
L
D
L
68
5
Tottenham
38
20
6
12
74:61
L
L
W
L
W
66
6
Chelsea
38
18
9
11
77:63
W
W
W
W
W
63
7
Newcastle
38
18
6
14
85:62
W
W
D
L
W
60
8
Man Utd
38
18
6
14
57:58
D
L
L
W
W
60
9
West Ham
38
14
10
14
60:74
L
D
L
W
L
52
10
Crystal Palace
38
13
10
15
57:58
W
D
W
W
W
49
11
Brighton
38
12
12
14
55:62
L
W
D
L
L
48
12
Bournemouth
38
13
9
16
54:67
W
W
L
L
L
48
13
Fulham
38
13
8
17
55:61
L
D
D
L
W
47
14
Wolves
38
13
7
18
50:65
L
W
L
L
L
46
15
Everton
38
13
9
16
40:51
W
W
D
W
L
40
16
Brentford
38
10
9
19
56:65
W
L
D
W
L
39
17
Forest
38
9
9
20
49:67
L
L
W
L
W
32
18
Luton
38
6
8
24
52:85
L
L
D
L
L
26
19
Burnley
38
5
9
24
41:78
W
D
L
L
L
24
20
Sheffield Utd
38
3
7
28
35:104
L
L
L
L
L
16


In [ ]:
driver.quit()

In [5]:
league_table_results_container

[]

In [6]:
data_copy = league_table_results_container

df_columns = data_copy[0][11:43]

data = data_copy[0][43:]

data

IndexError: list index out of range

In [ ]:
data_copy_split = [data.split('\n')]


data_copy_split = data_copy_split[0][1:]
print(data_copy_split)

In [ ]:
data_array = [data_copy_split[i:i+13] for i in range(0, len(data_copy_split), 13)]
print(data_array)

In [ ]:
testing = ['1', 'Arsenal', '36', '26', '5', '5', '88:28', 'L', 'W', 'W', 'W', 'W', '83']

# Splitting the element at index 6 and extending the list with the resulting parts
testing[6:7] = testing[6].split(':')

print(testing)

In [ ]:
split_data = []

for data in data_array:
    data[6:7] = data[6].split(':')
    split_data.append(data)    

print(split_data) 

In [ ]:
# These team_labels match all the data from the other notebooks
team_labels = {
        'Arsenal': 1,
        'Aston Villa': 2,
        'Bournemouth': 3,
        'Brighton': 4,
        'Burnley': 5,
        'Chelsea': 6,
        'Crystal Palace': 7,
        'Everton': 8,
        'Fulham': 9,
        'Leeds Utd': 10,
        'Leicester City': 11,
        'Liverpool': 12,
        'Man City': 13,
        'Man Utd': 14,
        'Newcastle': 15,
        'Norwich': 16,
        'Sheffield': 17,
        'Southampton': 18,
        'Tottenham': 19,
        'West Ham': 20,
        'Luton': 21,
        'Wolves': 22,
        'Brentford': 23,
        'Sheffield Utd': 24,
        'Forest': 25
    }

#### Code Playground: creating ppg, last_5_matches, and removing redundant indexes

In [ ]:
epl_table = [['1','Arsenal','37','27','5','5','89','28','W','W','W', 'W','W','86'],
 ['2','Man City', '36','26','7','3','91','33','W','W','W','W','W','85'],
 ['3','Liverpool','36','23','9','4','81','38','L','W','L','D','W','78']]

for row in epl_table:
    ppg = 0
    for result in row[7:13]:
        if result == 'W':
            ppg += 3
        elif result == 'D':
            ppg += 1
    last_5_matches = ''.join(row[8:13])  # Concatenate match results from index 7 to 11
    row[8:13] = [last_5_matches, ppg / 5]  # Replace old match results with last 5 matches string and PPG

print(epl_table)


In [ ]:
def calculate_ppg_and_create_last_5_matches(two_d_array):
    """
    Calculate the points per game (PPG) and create a string representing the last 5 matches for each team.

    Args:
        two_d_array (list of lists): Two-dimensional array containing match data for each team.

    Returns:
        list of lists: Updated two-dimensional array with additional columns for last 5 matches and PPG.
    """
    for row in two_d_array:
        ppg = 0
        for result in row[7:13]:
            if result == 'W':
                ppg += 3
            elif result == 'D':
                ppg += 1
        last_5_matches = ''.join(row[8:13])  # Concatenate match results from index 7 to 11
        row[8:13] = [last_5_matches, ppg / 5]  # Replace old match results with last 5 matches string and PPG
    
    return two_d_array

In [ ]:
premier_league_columns  = ['Pos', 'Team', 'Pld', 'Wins', 'Draws', 'Losses', 'GF', 'GA', 'Last 5 Matches', 'Ppg', 'Points']


In [ ]:
df_data = calculate_ppg_and_create_last_5_matches(split_data)


In [ ]:
import pandas as pd
# Create DataFrame
premier_league_table = pd.DataFrame(df_data, columns=premier_league_columns)
print(premier_league_table)


In [ ]:
def team_to_label(team_name):
    return team_labels.get(team_name)

In [ ]:
premier_league_table['Team'] = premier_league_table['Team'].map(team_to_label)

#### Confirm its the right Weekly Round in order to join with the right dataset. 
Note this data will eb considered as 'Previous League standings'(The data we are scrapping will be vital before the weekly fixtures | completed_matches in our case. Because we need the stats before the matches are played

In [ ]:
Round  = 'Round ' + ' ' +  premier_league_table['Pld'].mode()[0]

Establish your DB 
Make sure you have already created a database ready for it. In my next update i will have automations for creating the databse for you

In [ ]:
# Database connection
user = '<user>'
password = '<password>'
host = 'localhost'
port = 3306
database = '<db>'

In [ ]:
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')
premier_league_table.to_sql('previous_week_league_standings', con=engine, if_exists='replace', index=False)